In [3]:
from glob import glob
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

import my_module

pd.set_option("display.max_columns", None)

## Dataの読み込み
- df_care: train data
- df_care_test: test data
- df_acc: train & test data

In [4]:
df_acc, df_care_train, df_care_test = my_module.read_all_data()

## 前処理
- 欠損値は落とす
- 重複データは落とす
- hour_lengthがマイナスのデータは落とす
- trainの3月を含むそれ以降のデータは削除してもよさそう(heatmapより偏りがひどいため)
- df_accはdatetime、df_careはstartカラムでsort

In [5]:
# str convert to datetime
df_care_train = my_module.convert_datetime(df_care_train, ["start", "finish"])
df_care_test = my_module.convert_datetime(df_care_test, ["start", "finish"])
df_acc = my_module.convert_datetime(df_acc, ["datetime"])

# add time_length columns
df_care_train = my_module.add_timeLength_timeLengthSeconds(df_care_train)
df_care_test = my_module.add_timeLength_timeLengthSeconds(df_care_test)

# drop missing and duplicated data
df_care_train.dropna(inplace=True)
df_care_test.dropna(inplace=True)
df_acc.drop_duplicates(inplace=True)

# finish - start <= 0 のデータは処理の関係上落とす
df_care_train = df_care_train[df_care_train["time_length_seconds"] > 0]
df_care_test = df_care_test[df_care_test["time_length_seconds"] > 0]

# trainデータの4月を含む後のデータを抽出
df_care_train = df_care_train[df_care_train["start"] >= pd.Timestamp("2018-04-01T00:00:00.000+09:00")]
df_acc = df_acc[df_acc["datetime"] >= pd.Timestamp("2018-04-01T00:00:00.000+09:00")]

# datetimeごとにsort
df_care_train.sort_values("start", inplace=True)
df_care_test.sort_values("start", inplace=True)
df_acc.sort_values("datetime", inplace=True)

# ラベルが起こった時刻(hour特徴量)の追加
df_care_train["hour"] = df_care_train["year-month-date-hour"].str.split(pat="-", expand=True).iloc[:, -1].astype(float)
df_care_test["hour"] = df_care_test["year-month-date-hour"].str.split(pat="-", expand=True).iloc[:, -1].astype(float)

# # extend time (data argument)
# extend_time = 20 #20minute
# df_care_train = my_module.extend_time(df_care_train, extend_time)

## k-meansでクラスタリングを行う
- 普通のクラスタリング
- frequencyのmin-max正規化したもののクラスタリング
- 各userの総時間で割ったもののクラスタリング

In [6]:
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.cluster import SpectralClustering

users_heatmap = my_module.create_frequency_heatmap(df_care_train)

users_heatmap_dataframe = pd.DataFrame(users_heatmap).T
users_heatmap_dataframe.sort_index(inplace=True)

# default
kmeans = KMeans(n_clusters=3, max_iter=5)
cluster = kmeans.fit(users_heatmap_dataframe)

# min-max scaling
kmeans = KMeans(n_clusters=3, max_iter=5)
users_heatmap_dataframe_scaling = preprocessing.minmax_scale(users_heatmap_dataframe, axis=1)
cluster_min_max_scaling = kmeans.fit(users_heatmap_dataframe_scaling)

# time scaling
kmeans = KMeans(n_clusters=3, max_iter=5)
users_total_time = df_care_train.groupby(["user_id"])["time_length_seconds"].sum()
users_heatmap_time_scaling = users_heatmap_dataframe / users_total_time.values.reshape(5, 1)
cluster_time_scaling = kmeans.fit(users_heatmap_time_scaling)

print(users_heatmap_dataframe.index)
print(f"default: {cluster.labels_}")
print(f"min-max scale: {cluster_min_max_scaling.labels_}")
print(f"time scale: {cluster_time_scaling.labels_}")

Int64Index([8, 13, 14, 15, 25], dtype='int64')
default: [2 0 1 0 0]
min-max scale: [2 1 1 1 0]
time scale: [0 1 2 2 1]


## クラスタリングを元にheatmapの平均化
- time scaleを軸に考えると
    - クラス1: 8
    - クラス2: 13, 25
    - クラス3: 14, 15

In [7]:
df_care_all = pd.concat([df_care_train, df_care_test])

users_heatmap_train = my_module.create_frequency_heatmap(df_care_train)
users_heatmap_all = my_module.create_frequency_heatmap(df_care_all)


cluster_ave_heatmap ={}

users_heatmap = {index: data.reshape(28, 24) for index, data in users_heatmap_all.items()}

# class1
cluster_ave_heatmap[1] = users_heatmap[8]

# class2
cluster_ave_heatmap[2] = (users_heatmap[13] + users_heatmap[25]) / 2

# class3
cluster_ave_heatmap[3] = (users_heatmap[14] + users_heatmap[15]) / 2

## 各userで予測するactivity_labelのフィルタを設定
- user8 :   1,2,3,4,5,7,8,9,10,11,12,13,14,16,18,19,20,21,22,23,24
- user13:   1,2,4,7,8,9,10,11,12,13,14,15,16,19,22,25,27
- user14:   1,2,3,4,5,6,7,9,10,11,12,13,14,15,16,19
- user15:   1,2,3,4,5,9,10,11,12,13,14,16,18,19,22
- user25:   1,2,3,4,6,7,8,10,13,14,16,17,18,22,23,24,25,26

In [8]:
users_filters = my_module.create_users_filters()

## Validationデータの予測

### 正解ラベルの作成

In [9]:
df_care_train_y = my_module.create_y_label(df_care_train)
df_care_test_y = my_module.create_y_label(df_care_test)
df_care_all_y = my_module.create_y_label(df_care_all)
display(df_care_all_y)

,user_id,year-month-date-hour,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28
0,8,2018-04-01-8.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33,15,2018-04-02-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34,15,2018-04-02-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103,13,2018-04-02-10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
140,13,2018-04-02-11.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14741,15,2018-07-02-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14758,15,2018-07-02-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14770,15,2018-07-02-5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14780,15,2018-07-02-6.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 対応ヒートマップから予測

In [10]:
class_cluster_map = {
    8 : 1,
    13: 2,
    14: 3,
    15: 3,
    25: 2,
}

def create_predict_df(df_care_y, class_cluster_map):
    predict_list = [] 

    for _, rows in df_care_y.iterrows():
        user_id, date = rows["user_id"], rows["year-month-date-hour"]
        hour = int(float(date.rsplit("-")[-1]))
        cluster_index = class_cluster_map[user_id]
        user_heatmap = cluster_ave_heatmap[cluster_index]
        predict_list.append([user_id, date, *user_heatmap[:, hour]])
    df_predict = pd.DataFrame(predict_list, columns=["user_id", "year-month-date-hour", *np.arange(1, 29)])
    return df_predict

df_care_train_predict = create_predict_df(df_care_train_y, class_cluster_map)
df_care_test_predict = create_predict_df(df_care_test_y, class_cluster_map)
df_care_all_predict = create_predict_df(df_care_all_y, class_cluster_map)

display(df_care_train_predict)
display(df_care_test_predict)

,user_id,year-month-date-hour,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28
0,8,2018-04-01-8.0,13.0,60.0,0.0,6.0,0.0,0.0,7.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15,2018-04-02-0.0,0.0,0.5,0.0,55.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,263.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15,2018-04-02-1.0,0.0,0.0,0.0,24.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,272.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13,2018-04-02-10.0,1.0,0.0,0.0,10.5,0.0,16.5,5.0,4.5,0.0,80.0,0.0,0.0,2.0,1.5,0.0,1.5,1.5,1.0,0.0,0.0,0.0,2.5,0.0,7.5,1.5,0.0,0.0,0.0
4,13,2018-04-02-11.0,1.0,0.0,0.5,24.5,0.0,41.5,0.0,0.0,0.0,3.5,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.5,0.0,0.0,0.5,0.0,1.0,1.0,0.0,0.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,8,2018-05-23-19.0,4.0,125.0,23.0,96.0,0.0,0.0,0.0,1.0,0.0,0.0,42.0,19.0,4.0,0.0,0.0,1.0,0.0,3.0,69.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
708,8,2018-05-23-20.0,0.0,77.0,7.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,64.0,2.0,1.0,0.0,0.0,0.0,0.0,19.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
709,8,2018-05-23-21.0,0.0,39.0,5.0,76.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,185.0,0.0,0.0,0.0,7.0,0.0,0.0,8.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
710,8,2018-05-23-22.0,0.0,25.0,10.0,66.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,13.0,0.0,0.0,0.0,2.0,1.0,4.0,20.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


,user_id,year-month-date-hour,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28
0,14,2018-05-18-0.0,0.0,0.5,0.0,55.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,263.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,14,2018-05-18-1.0,0.0,0.0,0.0,24.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,272.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,14,2018-05-18-2.0,0.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,118.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,14,2018-05-18-3.0,0.0,0.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,232.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,14,2018-05-18-4.0,6.5,0.0,0.0,21.5,0.0,0.5,0.0,0.0,1.5,0.0,0.0,47.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,15,2018-07-02-3.0,0.0,0.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,232.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537,15,2018-07-02-4.0,6.5,0.0,0.0,21.5,0.0,0.5,0.0,0.0,1.5,0.0,0.0,47.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
538,15,2018-07-02-5.0,51.0,1.5,0.0,44.0,0.0,3.5,0.0,0.0,13.5,0.0,0.0,323.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
539,15,2018-07-02-6.0,44.0,0.0,1.5,30.0,0.0,6.0,0.0,0.0,29.0,0.0,0.0,4.5,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 評価

In [13]:
from sklearn.metrics import classification_report, accuracy_score
import warnings

y_true = df_care_all_y.iloc[:, 2:]
y_true[y_true>0] = 1
y_true[y_true<=0] = 0
y_true = y_true.astype(int)
y_pred = df_care_all_predict.iloc[:, 2:]
y_pred[y_pred >= 0.5] = 1
y_pred[y_pred < 0.5] = 0
y_pred = y_pred.astype(int)


print('accuracy: {0} %'.format(100*accuracy_score(y_true, y_pred)))

warnings.simplefilter("ignore")
target_names = [f"label_{i}" for i in range(1, 29)]
report = classification_report(y_pred, y_true, target_names=target_names, output_dict=True)
report_df = pd.DataFrame(report).T
display(report_df)

accuracy: 3.8180341186027618 %


,precision,recall,f1-score,support
label_1,0.921687,0.203457,0.333333,752.0
label_2,0.975000,0.388543,0.555654,803.0
label_3,0.898305,0.143631,0.247664,369.0
label_4,1.000000,0.508530,0.674206,1231.0
label_5,0.882353,0.107143,0.191083,140.0
label_6,0.931973,0.313501,0.469178,437.0
label_7,1.000000,0.099379,0.180791,161.0
label_8,0.923077,0.061538,0.115385,195.0
label_9,0.980583,0.331148,0.495098,305.0
label_10,1.000000,0.262745,0.416149,510.0


In [14]:
import dataframe_image as dfi
report_df.dfi.export("report.png")

[0727/095714.099918:ERROR:bus.cc(398)] Failed to connect to the bus: Failed to connect to socket /var/run/dbus/system_bus_socket: No such file or directory
[0727/095714.099992:ERROR:bus.cc(398)] Failed to connect to the bus: Failed to connect to socket /var/run/dbus/system_bus_socket: No such file or directory
[0727/095714.101278:WARNING:bluez_dbus_manager.cc(247)] Floss manager not present, cannot set Floss enable/disable.
[0727/095714.110854:WARNING:sandbox_linux.cc(376)] InitializeSandbox() called with multiple threads in process gpu-process.
[0727/095714.258451:INFO:headless_shell.cc(660)] Written to file /tmp/tmpbctu3vw2/temp.png.
[0727/095714.370808:ERROR:bus.cc(398)] Failed to connect to the bus: Failed to connect to socket /var/run/dbus/system_bus_socket: No such file or directory
[0727/095714.370906:ERROR:bus.cc(398)] Failed to connect to the bus: Failed to connect to socket /var/run/dbus/system_bus_socket: No such file or directory
[0727/095714.372168:WARNING:bluez_dbus_manag

### accデータのラベリング
- 後々使うかもしれないので各ユーザーごとに使用できるaccデータを結びつける

In [ ]:
seg_list, seg_label_list = my_module.create_acc_dataframe_label(df_care_test, df_acc)

In [ ]:
for k, v in seg_list.items():
    print(f"user{k}: {len(v)}")
for k, v in seg_label_list.items():
    # print(k, v)
    print(f"user{k}: {set(v)}")

user14: 464
user13: 146
user25: 8
user15: 0
user8: 54
user14: {1, 2, 3, 4, 9, 11, 12, 13, 16}
user13: {1, 2, 4, 9, 10, 11, 12, 14, 16}
user25: {24, 18, 10, 6}
user15: set()
user8: {1, 2, 4, 7, 8, 9, 10, 12, 13, 19, 20}


### 各時間に行われたactivityをtypeごとに集計して特徴量として追加する

In [ ]:
# 各時間に行われたactivityの集計
def makeFeatures(df):
    feat = df.groupby(['activity_type_id', 'year-month-date-hour'], as_index=False).count()[['activity_type_id', 'year-month-date-hour','id']].rename(columns={'id':'count'})
    X = df_care_train.copy()
    print(feat)
    acts = list(df['activity_type_id'].value_counts().index)
    for act in acts:
        X = pd.merge(X, feat[feat['activity_type_id'] == act][['year-month-date-hour', 'count']], on='year-month-date-hour', how="left").rename(columns={'count':f"label_{act}"})
    return X
df_care_feature = makeFeatures(df_care_train)
#TODO: ここのエラーを解消
# df_care_feature = df_care_feature.fillna(0.0)

      activity_type_id year-month-date-hour  count
0                    1       2018-03-01-9.0     12
1                    1      2018-03-06-17.0     16
2                    1      2018-03-08-16.0      1
3                    1      2018-03-11-19.0      1
4                    1      2018-03-11-20.0      1
...                ...                  ...    ...
2033                27      2018-05-13-15.0      1
2034                27      2018-05-23-19.0      1
2035                28      2018-02-07-17.0      1
2036                28      2018-03-14-19.0      1
2037                28      2018-05-12-11.0      1

[2038 rows x 3 columns]


### 時間とactivityごとに集計したトータル

In [ ]:
df_care_feature["hour"] = df_care_feature["year-month-date-hour"].str.split("-", expand=True).iloc[:, -1].astype(float)
corr = df_care_feature.groupby(["activity_type_id", "hour"]).count().iloc[:, 0].reset_index().sort_values(["activity_type_id", "hour"]).pivot("activity_type_id", "hour").fillna(0).astype(int)

In [ ]:
# ここに特徴量を追加する
tmp = df_care_feature.groupby(["hour", "activity_type_id"]).count().iloc[:, 0].reset_index()
tmp[tmp["hour"] == 0.0]

,hour,activity_type_id,id
0,0.0,2,1
1,0.0,4,150
2,0.0,11,17
3,0.0,12,504
4,0.0,17,1
5,0.0,19,4
6,0.0,23,3


In [ ]:
df_tmp = df_care_feature.copy()
activities = df_tmp["activity_type_id"].unique()
for activity in activities:
    df_tmp[f"{activity}_total"] = 0

In [ ]:
df_care_feature.head()

,id,user_id,activity_type_id,activity_type,target_id,activity2user_id,start,finish,year-month-date-hour,time_length,time_length_seconds,hour,12,2,4,10,1,11,19,6,9,7,5,16,3,18,17,20,24,13,8,26,22,14,23,21,25,27,15,28
0,179,14,18,申し送り・ミーティング,3,14624,2018-02-02 09:47:16+09:00,2018-02-02 09:47:21+09:00,2018-02-02-9.0,0 days 00:00:05,5.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,187,14,10,日中利用者対応,35,12354,2018-02-02 11:12:03+09:00,2018-02-02 11:09:40+09:00,2018-02-02-11.0,-1 days +23:57:37,-143.0,11.0,0.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,189,14,17,手書き記録,31,11803,2018-02-02 11:27:59+09:00,2018-02-02 11:28:00+09:00,2018-02-02-11.0,0 days 00:00:01,1.0,11.0,0.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,189,14,17,手書き記録,35,11805,2018-02-02 11:27:59+09:00,2018-02-02 11:28:00+09:00,2018-02-02-11.0,0 days 00:00:01,1.0,11.0,0.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,189,14,17,手書き記録,36,11806,2018-02-02 11:27:59+09:00,2018-02-02 11:28:00+09:00,2018-02-02-11.0,0 days 00:00:01,1.0,11.0,0.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
